In [1]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 346 kB 5.2 MB/s 
     |████████████████████████████████| 86 kB 7.6 MB/s 
     |████████████████████████████████| 86 kB 6.8 MB/s 
     |████████████████████████████████| 1.1 MB 63.8 MB/s 
     |████████████████████████████████| 140 kB 19.3 MB/s 
     |████████████████████████████████| 212 kB 67.1 MB/s 
     |████████████████████████████████| 596 kB 57.5 MB/s 
     |████████████████████████████████| 127 kB 72.3 MB/s 
     |████████████████████████████████| 94 kB 4.0 MB/s 
     |████████████████████████████████| 271 kB 72.9 MB/s 
     |████████████████████████████████| 144 kB 69.1 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
     

In [2]:
!pip install enum34

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
!pip install zstandard

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.5 MB 5.4 MB/s 


In [4]:
import math
import time

import torch
import torch.nn as nn
import torch.optim as optim
from torch.nn import Parameter

import torchtext

import datasets
from enum import IntEnum
class Dim(IntEnum):
    batch = 0
    seq = 1
    feature = 2

In [5]:
#torch.manual_seed(0)

In [6]:
dataset = datasets.load_dataset('ptb_text_only')

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/42068 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3761 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3370 [00:00<?, ? examples/s]

Dataset ptb_text_only downloaded and prepared to /root/.cache/huggingface/datasets/ptb_text_only/penn_treebank/1.1.0/8d1b97746fb9765d140e569ec5ddd35e20af4d37761f5e1bf357ea0b081f2c1f. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [7]:
dataset

DatasetDict({
    train: Dataset({
        features: ['sentence'],
        num_rows: 42068
    })
    test: Dataset({
        features: ['sentence'],
        num_rows: 3761
    })
    validation: Dataset({
        features: ['sentence'],
        num_rows: 3370
    })
})

In [8]:
dataset['train'][0]

{'sentence': 'aer banknote berlitz calloway centrust cluett fromstein gitano guterman hydro-quebec ipo kia memotec mlx nahb punts rake regatta rubens sim snack-food ssangyong swapo wachter'}

In [9]:
dataset['train'][1]

{'sentence': 'pierre <unk> N years old will join the board as a nonexecutive director nov. N'}

In [10]:
tokenizer = torchtext.data.utils.get_tokenizer('basic_english')

In [11]:
tokenizer('hello world how are you?')

['hello', 'world', 'how', 'are', 'you', '?']

In [12]:
tokenizer(dataset['train'][1]['sentence'])

['pierre',
 '<unk>',
 'n',
 'years',
 'old',
 'will',
 'join',
 'the',
 'board',
 'as',
 'a',
 'nonexecutive',
 'director',
 'nov',
 '.',
 'n']

In [13]:
def tokenize_data(example, tokenizer):
    tokens = {'tokens': tokenizer(example['sentence'])}
    return tokens

In [14]:
tokenized_dataset = dataset.map(tokenize_data, remove_columns=['sentence'], fn_kwargs={'tokenizer': tokenizer})

  0%|          | 0/42068 [00:00<?, ?ex/s]

  0%|          | 0/3761 [00:00<?, ?ex/s]

  0%|          | 0/3370 [00:00<?, ?ex/s]

In [15]:
tokenized_dataset['train'][1]

{'tokens': ['pierre',
  '<unk>',
  'n',
  'years',
  'old',
  'will',
  'join',
  'the',
  'board',
  'as',
  'a',
  'nonexecutive',
  'director',
  'nov',
  '.',
  'n']}

In [16]:
vocab = torchtext.vocab.build_vocab_from_iterator(tokenized_dataset['train']['tokens'],
                                                  min_freq=3)

In [17]:
vocab.get_itos()[:10]

['the', '<unk>', 'n', 'of', 'to', 'a', 'in', 'and', '.', "'"]

In [18]:
len(vocab)

9878

In [19]:
'hello' in vocab

False

In [20]:
#vocab.insert_token('<unk>', 0)

In [21]:
vocab.get_itos()[:10]

['the', '<unk>', 'n', 'of', 'to', 'a', 'in', 'and', '.', "'"]

In [22]:
unk_index = vocab['<unk>']
vocab.set_default_index(unk_index)

In [23]:
vocab['hello']

1

In [24]:
vocab.insert_token('<eos>', 1)

In [25]:
vocab.get_itos()[:10]

['the', '<eos>', '<unk>', 'n', 'of', 'to', 'a', 'in', 'and', '.']

In [26]:
def get_data(dataset, vocab, batch_size):
    data = []
    for example in dataset:
        if example['tokens']:
            tokens = example['tokens'].append('<eos>')
            tokens = [vocab[token] for token in example['tokens']]
            data.extend(tokens)
    data = torch.LongTensor(data)
    n_batches = data.shape[0] // batch_size
    data = data.narrow(0, 0, n_batches * batch_size)
    data = data.view(batch_size, -1)
    return data

In [27]:
batch_size = 64

train_data = get_data(tokenized_dataset['train'], vocab, batch_size)

In [28]:
train_data.shape

torch.Size([64, 15101])

In [29]:
valid_data = get_data(tokenized_dataset['validation'], vocab, batch_size)
test_data = get_data(tokenized_dataset['test'], vocab, batch_size)

In [30]:
class MogLSTM(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_sz, n_layers, dropout_rate, tie_weights,mog_iterations):
        super().__init__()
        self.n_layers = n_layers
        self.hidden_dim = hidden_sz
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        #self.lstm = nn.LSTM(embedding_dim, hidden_dim, num_layers=n_layers, dropout=dropout_rate, batch_first=True)
        self.fc = nn.Linear(hidden_dim, vocab_size)
        self.dropoutO = nn.Dropout(dropout_rate[1])
        self.dropoutI = nn.Dropout(dropout_rate[2])
        self.dropoutH = nn.Dropout(dropout_rate[3])
        self.dropoutE = nn.Dropout(dropout_rate[0])


        self.mog_iterations = mog_iterations
        #Define/initialize all tensors   
        self.Wih = Parameter(torch.Tensor(embedding_dim, hidden_sz * 4))
        self.Whh = Parameter(torch.Tensor(hidden_sz, hidden_sz * 4))
        self.bih = Parameter(torch.Tensor(hidden_sz * 4))
        self.bhh = Parameter(torch.Tensor(hidden_sz * 4))
        #Mogrifiers
        self.Q = Parameter(torch.Tensor(hidden_sz,embedding_dim))
        self.R = Parameter(torch.Tensor(embedding_dim,hidden_sz))


        if tie_weights:
            assert embedding_dim == hidden_dim, 'If tying weights then embedding_dim must equal hidden_dim'
            self.embedding.weight = self.fc.weight

        self.init_weights()
    
    def init_weights(self):
        for p in self.parameters():
            if p.data.ndimension() >= 2:
                nn.init.xavier_uniform_(p.data)
            else:
                nn.init.zeros_(p.data)


    def init_hidden(self, batch_size, device):
        hidden = torch.zeros( batch_size, self.hidden_dim).to(device)
        cell = torch.zeros( batch_size, self.hidden_dim).to(device)
        return (hidden,cell)

    def detach_hidden(self, hidden):
       hidden, cell = hidden
       hidden = hidden.detach()
       cell = cell.detach()
       return hidden, cell

    def mogrify(self,xt,ht):
      for i in range(1,self.mog_iterations+1):
        if (i % 2 == 0):
          ht = (2*torch.sigmoid(xt @ self.R)) * ht
        else:
          xt = (2*torch.sigmoid(ht @ self.Q)) * xt
      return xt, ht
    
    def moglstm(self, x: torch.Tensor, 
                hidden:torch.Tensor,cell:torch.Tensor):
        """Assumes x is of shape (batch, sequence, feature)"""
        batch_sz, seq_sz, _ = x.size()
        hidden_seq = []
        #ht and Ct start as the previous states and end as the output states in each loop below
       # if init_states is None:
        #    ht = torch.zeros((batch_sz,self.hidden_size)).to(x.device)
         #   Ct = torch.zeros((batch_sz,self.hidden_size)).to(x.device)
        #else:
        ht, Ct = hidden, cell
        for t in range(seq_sz): # iterate over the time steps
            xt = x[:, t, :]
            xt = self.dropoutI(xt)
            ht = self.dropoutH(ht)
            xt, ht = self.mogrify(xt,ht) #mogrification
            gates = (xt @ self.Wih + self.bih) + (ht @ self.Whh + self.bhh)
            ingate, forgetgate, cellgate, outgate = gates.chunk(4, 1)

            ### The LSTM Cell!
            ft = torch.sigmoid(forgetgate)
            it = torch.sigmoid(ingate)
            Ct_candidate = torch.tanh(cellgate)
            ot = torch.sigmoid(outgate)
            #outputs
            Ct = (ft * Ct) + (it * Ct_candidate)
            ht = ot * torch.tanh(Ct)
            ###

            hidden_seq.append(ht.unsqueeze(Dim.batch))
        hidden_seq = torch.cat(hidden_seq, dim=Dim.batch)
        # reshape from shape (sequence, batch, feature) to (batch, sequence, feature)
        hidden_seq = hidden_seq.transpose(Dim.batch, Dim.seq).contiguous()
        return hidden_seq, (ht, Ct)


    def forward(self, input,hidden,cell):
        # input = [batch size, seq len]
        # hidden = [n layers, batch size, hidden dim]
        embedding = self.dropoutE(self.embedding(input))
        # embedding = [batch size, seq len, embedding dim]
        output, hidden = self.moglstm(embedding, hidden ,cell)
        # output = [batch size, seq len, hidden dim]
        # hidden = [n layers, batch size, hidden dim]
        output = self.dropoutO(output)
        prediction = self.fc(output)
        # prediction = [batch size, seq len, vocab size]
        return prediction, hidden

In [48]:
vocab_size = len(vocab)
embedding_dim = 1170
hidden_dim = 1170
n_layers = 1
dropout_rate = [0.15,0.6,0.5,0.7]
tie_weights = True

model = MogLSTM(vocab_size, embedding_dim, hidden_dim, n_layers, dropout_rate, tie_weights,mog_iterations=2)

In [49]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 25,266,669 trainable parameters


In [50]:
lr = 1e-3

optimizer = optim.Adam(model.parameters(), lr=lr, betas=(0,0.999) )

In [51]:
criterion = nn.CrossEntropyLoss()

In [52]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print(device)

cuda


In [53]:
model = model.to(device)
criterion = criterion.to(device)

In [54]:
batch_size

64

In [55]:
def train(model, data, optimizer, criterion, batch_size, max_seq_len, clip, device):
    
    epoch_loss = 0
    model.train()
    n_tokens = data.shape[-1]

    hidden = model.init_hidden(batch_size, device)
    
    for offset in range(0, n_tokens - 1, max_seq_len):
        optimizer.zero_grad()
        input, target, seq_len = get_batch(data, max_seq_len, n_tokens, offset)
        input = input.to(device)
        target = target.to(device)
        # input = [batch size, seq len]
        # target = [batch size, seq len]
        batch_sz, seq_sz = input.shape
        hidden, cell = model.detach_hidden(hidden)
        #### hidden = [n layers, batch size, hidden dim]
        prediction, hidden = model(input, hidden, cell)
        # prediction = [batch size, seq len, vocab size]
        # hidden = [n layers, batch size, hidden dim]
        prediction = prediction.reshape(batch_size * seq_len, -1)
        target = target.reshape(-1)
        # prediction = [batch size * seq len, vocab size]
        # target = [batch size * seq len]
        loss = criterion(prediction, target)
        #lossl2 = loss + alpha*L2(model)/2*seq_len
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        optimizer.step()
        epoch_loss += loss.item() * seq_len
    return epoch_loss / n_tokens

In [56]:
hidden = model.init_hidden(batch_size,device)
hidden1, cell = model.detach_hidden(hidden)


In [57]:
hidden1.shape

torch.Size([64, 1170])

In [58]:
def get_batch(data, max_seq_len, n_tokens, offset):
    seq_len = min(max_seq_len, n_tokens - offset - 1)
    input = data[:, offset:offset+seq_len]
    target = data[:, offset+1:offset+seq_len+1]
    return input, target, seq_len

In [59]:
def evaluate(model, data, criterion, batch_size, max_seq_len, device):

    epoch_loss = 0
    model.eval()
    n_tokens = data.shape[-1]

    hidden = model.init_hidden(batch_size, device)

    with torch.no_grad():
        for offset in range(0, n_tokens - 1, max_seq_len):
            input, target, seq_len = get_batch(data, max_seq_len, n_tokens, offset)
            input = input.to(device)
            target = target.to(device)
            # input = [batch size, seq len]
            # target = [batch size, seq len]
            batch_sz, seq_sz = input.shape
            hidden,cell = model.detach_hidden(hidden)
            # hidden = [n layers, batch size, hidden dim]
            prediction, hidden = model(input, hidden,cell)
            # prediction = [batch size, seq len, vocab size]
            # hidden = [n layers, batch size, hidden dim]
            prediction = prediction.reshape(batch_size * seq_len, -1)
            target = target.reshape(-1)
            # prediction = [batch size * seq len, vocab size]
            # target = [batch size * seq len]
            loss = criterion(prediction, target)
            epoch_loss += loss.item() * seq_len
    return epoch_loss / n_tokens

In [60]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [61]:
lr_scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.5, patience=0)

In [62]:
max_seq_len=64

In [63]:
n_epochs = 50
max_seq_len = 50
clip = 0.25

best_valid_loss = float('inf')

for epoch in range(n_epochs):

    start_time = time.monotonic()

    train_loss = train(model, train_data, optimizer, criterion, batch_size, max_seq_len, clip, device)
    valid_loss = evaluate(model, valid_data, criterion, batch_size, max_seq_len, device)
    
    lr_scheduler.step(valid_loss)

    end_time = time.monotonic()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)

    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'lstm_lm.pt')

    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Perplexity: {math.exp(train_loss):.3f}')
    print(f'\tValid Perplexity: {math.exp(valid_loss):.3f}')

Epoch: 01 | Epoch Time: 1m 10s
	Train Perplexity: 333.180
	Valid Perplexity: 257.007
Epoch: 02 | Epoch Time: 1m 12s
	Train Perplexity: 163.962
	Valid Perplexity: 130.930
Epoch: 03 | Epoch Time: 1m 12s
	Train Perplexity: 125.487
	Valid Perplexity: 110.937
Epoch: 04 | Epoch Time: 1m 13s
	Train Perplexity: 105.242
	Valid Perplexity: 99.203
Epoch: 05 | Epoch Time: 1m 13s
	Train Perplexity: 92.029
	Valid Perplexity: 91.365
Epoch: 06 | Epoch Time: 1m 14s
	Train Perplexity: 82.649
	Valid Perplexity: 86.174
Epoch: 07 | Epoch Time: 1m 14s
	Train Perplexity: 75.760
	Valid Perplexity: 82.878
Epoch: 08 | Epoch Time: 1m 14s
	Train Perplexity: 70.287
	Valid Perplexity: 80.392
Epoch: 09 | Epoch Time: 1m 14s
	Train Perplexity: 65.931
	Valid Perplexity: 78.356
Epoch: 10 | Epoch Time: 1m 14s
	Train Perplexity: 62.413
	Valid Perplexity: 76.849
Epoch: 11 | Epoch Time: 1m 14s
	Train Perplexity: 59.323
	Valid Perplexity: 75.727
Epoch: 12 | Epoch Time: 1m 14s
	Train Perplexity: 56.761
	Valid Perplexity: 74.8

KeyboardInterrupt: ignored

In [ ]:
model.load_state_dict(torch.load('lstm_lm.pt'))

test_loss = evaluate(model, test_data, criterion, batch_size, max_seq_len, device)

print(f'Test Perplexity: {math.exp(test_loss):.3f}')

Test Perplexity: 69.251


In [ ]:
def generate(prompt, n_gen_tokens, temperature, model, tokenizer, vocab, device, seed=None):
    if seed is not None:
        torch.manual_seed(0)
    model.eval()
    tokens = tokenizer(prompt)
    indices = [vocab[t] for t in tokens]
    batch_size = 1
    hidden = model.init_hidden(batch_size, device)
    with torch.no_grad():
        for i in range(n_gen_tokens):
            input = torch.LongTensor([indices]).to(device)
            prediction, hidden = model(input, hidden)
            probs = torch.softmax(prediction[:, -1] / temperature, dim=-1) 
            prediction = torch.multinomial(probs, num_samples=1).item()
            indices.append(prediction)

    itos = vocab.get_itos()
    tokens = [itos[i] for i in indices]
    return tokens

In [ ]:
prompt = 'the'
n_gen_tokens = 25
temperature = 0.5
seed = 0

generation = generate(prompt, n_gen_tokens, temperature, model, tokenizer, vocab, device, seed)

In [ ]:
generation

In [ ]:
temperature = 0.1

generation = generate(prompt, n_gen_tokens, temperature, model, tokenizer, vocab, device, seed)

In [ ]:
generation

['the',
 '<unk>',
 '<unk>',
 ',',
 'which',
 'was',
 'the',
 'first',
 'to',
 'be',
 'built',
 'in',
 'the',
 '<unk>',
 '.',
 '<eos>',
 '=',
 '=',
 '=',
 '=',
 'chapel',
 'of',
 'our',
 'lady',
 'of',
 'our']

In [ ]:
temperature = 1.5

generation = generate(prompt, n_gen_tokens, temperature, model, tokenizer, vocab, device, seed)

In [ ]:
generation

['the',
 'hide',
 'swap',
 'just',
 'leads',
 'landmarks',
 'and',
 'arranged',
 'discussions',
 '3',
 'agree',
 'specifically',
 'with',
 'the',
 'friend',
 'harvest',
 'as',
 'captains',
 'like',
 'tom',
 'bradley',
 'giger',
 'viewed',
 'the',
 'team',
 "'"]

In [ ]:
temperature = 0.75

generation = generate(prompt, n_gen_tokens, temperature, model, tokenizer, vocab, device, seed)

In [ ]:
generation

['the',
 'highest',
 '<unk>',
 'in',
 'the',
 'united',
 'states',
 '.',
 'it',
 'is',
 'a',
 'oldman',
 'city',
 ',',
 'and',
 'the',
 'st',
 '.',
 'louis',
 'rail',
 'district',
 'has',
 'a',
 'population',
 'of',
 '17']

In [ ]:
temperature = 0.8

generation = generate(prompt, n_gen_tokens, temperature, model, tokenizer, vocab, device, seed)

In [ ]:
generation

['the',
 'highest',
 'swap',
 'in',
 'the',
 'era',
 '.',
 'the',
 'old',
 '3',
 '@',
 '.',
 '@',
 '06',
 'm',
 '(',
 '3',
 '@',
 '.',
 '@',
 '6',
 'ft',
 ')',
 'wide',
 ',',
 'fifth']

In [ ]:
temperature = 0.7

generation = generate(prompt, n_gen_tokens, temperature, model, tokenizer, vocab, device, seed)

In [ ]:
generation

['the',
 'highest',
 '<unk>',
 'in',
 'the',
 'united',
 'states',
 '.',
 'it',
 'is',
 'a',
 '<unk>',
 '@-@',
 '<unk>',
 'and',
 'a',
 '@-@',
 '<unk>',
 '@-@',
 'chorus',
 'sample',
 ',',
 'which',
 'features',
 'the',
 '<unk>']